# Soft Skills Assessment Notebook

This notebook creates a comprehensive soft skills assessment tool covering four key areas:
- Communication skills
- Leadership skills
- Time management skills
- Analytical skills

The assessment collects responses on a 5-point scale (strongly disagree to strongly agree) and provides feedback based on overall scores.

In [60]:
# Import necessary libraries
import requests
import re
import pandas as pd
import numpy as np
import os
import json
import random
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from IPython.display import display, HTML
import os
from dotenv import load_dotenv

## 1. Data Collection: Web Scraping

This section handles gathering questions from existing online assessment sources.
We scrape questions for each skill category from relevant HR and professional development websites.

In [61]:
# Create directories for saving data
os.makedirs('./data', exist_ok=True)

# Define the scraping function for all categories
def scrape_soft_skills_questions(category):
    """
    Scrape questions for a specific soft skill category from relevant websites.

    Args:
        category (str): One of 'communication', 'leadership', 'time_management', 'analytical'

    Returns:
        pd.DataFrame: DataFrame containing scraped questions
    """
    all_questions = []

    # Define target URLs and questions by category
    target_sources = {
        'communication': {
            'https://hr-survey.com/360_Survey_Example_1.htm': [
                "Takes on challenging questions and provides instant answers.",
                "Communicates clearly and gets to the point without unnecessary details.",
                "Coaches others on their written communication skills",
                "Addresses issues of key importance to stakeholders.",
                "Communicates goals of project, resources required, resources available, etc. to the team",
                "Articulates ideas and emotions clearly to others.",
                "Gives clear and convincing presentations."
            ],
            'https://hr-survey.com/360Feedback_Sample_Assessment_Form.htm': [
                "Adapts language and terminology to meet the needs of the audience.",
                "Recaps action steps from meetings to ensure clarity and execution.",
                "Considers the audience in how the communication is presented.",
                "Responds in a timely manner, respecting deadlines and others' schedules."
            ]
        },
        'leadership': {
            'https://hr-survey.com/360Leadership.htm': [
                "Sets a good example for the team to follow.",
                "Inspires others to achieve their best performance.",
                "Makes difficult decisions when necessary.",
                "Delegates tasks effectively to team members.",
                "Provides constructive feedback to help team members grow."
            ],
            'https://www.indeed.com/career-advice/career-development/leadership-assessment': [
                "Takes responsibility for team outcomes, both successes and failures.",
                "Recognizes and rewards team members' contributions.",
                "Adapts leadership style based on situation and team needs.",
                "Communicates a clear vision that motivates the team."
            ]
        },
        'time_management': {
            'https://www.mindtools.com/pages/article/newHTE_88.htm': [
                "Prioritizes tasks based on importance and deadlines.",
                "Completes work within allocated timeframes.",
                "Plans daily activities to maximize productivity.",
                "Avoids procrastination by breaking large tasks into smaller steps."
            ],
            'https://www.manager-tools.com/products/time-management-self-assessment': [
                "Maintains an organized workspace to minimize time spent searching for items.",
                "Sets realistic timeframes for completing tasks.",
                "Uses tools and technology to automate repetitive tasks.",
                "Effectively manages interruptions during focused work."
            ]
        },
        'analytical': {
            'https://www.mindtools.com/pages/article/newTMC_03.htm': [
                "Identifies patterns and trends in complex data sets.",
                "Breaks down problems into manageable components.",
                "Evaluates multiple solutions before making decisions.",
                "Distinguishes between correlation and causation in data analysis."
            ],
            'https://www.analyticssteps.com/blogs/analytical-skill-test-how-measure-analytical-skills': [
                "Uses logical reasoning to solve problems systematically.",
                "Gathers relevant information before drawing conclusions.",
                "Validates assumptions with appropriate evidence.",
                "Identifies potential biases in data interpretation."
            ]
        }
    }

    # Get target URLs and questions for the specified category
    if category not in target_sources:
        print(f"No scraping sources defined for category: {category}")
        return pd.DataFrame()

    category_sources = target_sources[category]

    # Scrape from each URL
    for url, questions in category_sources.items():
        try:
            print(f"Scraping {len(questions)} questions from {url}...")

            # Add headers to simulate a browser request
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }

            # Make the request with a timeout
            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code != 200:
                print(f"Failed to access {url} - Status code: {response.status_code}")
                continue

            html_content = response.text

            # Use regex to search for each question in the HTML
            for question in questions:
                # Escape special characters for regex
                escaped_question = re.escape(question)

                # If found in the HTML content, add to our list
                if re.search(escaped_question, html_content, re.IGNORECASE):
                    all_questions.append({
                        'question_text': question,
                        'source_url': url,
                        'category': category,
                        'type': 'scraped' # Mark as scraped data
                    })
                    print(f"Found question: {question[:50]}...")

            # Be nice to the servers
            time.sleep(1)

        except Exception as e:
            print(f"Error scraping {url}: {e}")

    # Create DataFrame from the collected questions
    df = pd.DataFrame(all_questions)

    # Debug info
    print(f"Total {category} questions scraped: {len(df)}")

    return df


## 2.Question Generation: Template-Based Approach

When we need more questions than we can scrape, this template-based generation system creates natural-sounding
assessment questions by combining sentence templates with category-specific components.

In [62]:
# Function for generating questions using template-based approach (instead of OpenAI)
def generate_questions_with_templates(category, n=20):
    """
    Generate questions using templates and components for a specific soft skill category.

    Args:
        category (str): One of 'communication', 'leadership', 'time_management', 'analytical'
        n (int): Number of questions to generate

    Returns:
        pd.DataFrame: DataFrame containing generated questions
    """
    generated_questions = []

    # Get templates and components for the category
    templates = get_question_templates(category)
    components = get_question_components(category)

    if not templates or not components:
        print(f"No templates or components defined for category: {category}")
        return pd.DataFrame()

    try:
        print(f"Generating {n} questions for {category} skills...")

        question_list = []
        attempts = 0
        max_attempts = n * 5 # To avoid infinite loops

        while len(question_list) < n and attempts < max_attempts:
            attempts += 1
            template = random.choice(templates)
            filled_question = fill_template_with_components(template, components)

            # Ensure uniqueness
            if filled_question not in question_list:
                question_list.append(filled_question)

        # Create data entries for each generated question
        for question in question_list:
            generated_questions.append({
                'question_text': question,
                'source_url': 'generated_by_templates',
                'category': category,
                'type': 'generated'
            })

    except Exception as e:
        print(f"Error generating questions with templates: {e}")

    # Create DataFrame from generated questions
    df = pd.DataFrame(generated_questions)

    # Debug info
    print(f"Successfully generated {len(df)} questions for {category} skills")

    return df


## 3.Question Generation: AI-Powered Approach

This alternative generation method uses the Hugging Face Inference API to create questions using advanced language models.
This produces more varied and nuanced questions but requires an API token.

In [ ]:
# Alternative: Use Hugging Face Inference API (free tier)
def generate_questions_with_huggingface(category, n=20):
    """
    Generate questions using Hugging Face's inference API for a specific soft skill category.

    Args:
        category (str): One of 'communication', 'leadership', 'time_management', 'analytical'
        n (int): Number of questions to generate

    Returns:
        pd.DataFrame: DataFrame containing generated questions
    """
    try:
        # Import the required libraries
        import requests

        # Define the prompt for each category
        category_prompts = {
            'communication': f"Generate {n} unique assessment questions for evaluating communication skills. Each question should be phrased as a statement that can be rated on a scale from 1 (strongly disagree) to 5 (strongly agree). Focus on professional communication in workplace settings.",
            'leadership': f"Generate {n} unique assessment questions for evaluating leadership skills. Each question should be phrased as a statement that can be rated on a scale from 1 (strongly disagree) to 5 (strongly agree). Cover different aspects of leadership including vision, motivation, delegation, and team development.",
            'time_management': f"Generate {n} unique assessment questions for evaluating time management skills. Each question should be phrased as a statement that can be rated on a scale from 1 (strongly disagree) to 5 (strongly agree). Include questions about prioritization, planning, efficiency, and avoiding procrastination.",
            'analytical': f"Generate {n} unique assessment questions for evaluating analytical skills. Each question should be phrased as a statement that can be rated on a scale from 1 (strongly disagree) to 5 (strongly agree). Cover aspects such as problem-solving, data analysis, logical reasoning, and critical thinking."
        }

        prompt = category_prompts.get(category, "")

        if not prompt:
            print(f"No prompt defined for category: {category}")
            # Fall back to template generation
            return generate_questions_with_templates(category, n)

        print(f"Generating {n} questions for {category} skills using Hugging Face...")

        # Load environment variables from .env file
        load_dotenv()
        # Access the token securely
        API_TOKEN = os.environ.get("HUGGINGFACE_API_TOKEN", "").strip()

        if not API_TOKEN or API_TOKEN == "your_token_here":
            print("💡 No valid API token found. Falling back to template-based generation.")
            print("To enable AI generation, add your Hugging Face token to the .env file")
            return generate_questions_with_templates(category, n)

        # Define the API endpoint (using a suitable model)
        API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-xxl" # A good free model

        # Set up the headers with your token
        headers = {
            "Authorization": f"Bearer {API_TOKEN}",
            "Content-Type": "application/json"
        }

        # Make the request
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 1024,
                "temperature": 0.7
            }
        }

        response = requests.post(API_URL, headers=headers, json=payload)

        # Parse the response with improved error handling
        if response.status_code == 200:
            result = response.json()

            # Extract the generated text
            if isinstance(result, list) and len(result) > 0 and "generated_text" in result[0]:
                generated_text = result[0]["generated_text"]
            else:
                generated_text = str(result)

            # Split into individual questions (assuming each is on a new line)
            raw_questions = [q.strip() for q in generated_text.split('\n') if q.strip()]

            # Clean up and format the questions
            questions = []
            for q in raw_questions:
                # Remove numbering if present
                if re.match(r'^\d+\.?\s+', q):
                    q = re.sub(r'^\d+\.?\s+', '', q)
                questions.append(q)

            # Limit to the requested number
            questions = questions[:n]

            # Create DataFrame entries
            df_data = [{
                'question_text': question,
                'source_url': 'generated_by_huggingface',
                'category': category,
                'type': 'generated'
            } for question in questions]

            return pd.DataFrame(df_data)
        elif response.status_code == 401:
            print("🔐 Authentication Error: Invalid Hugging Face API token.")
            print("Please check your token in the .env file and try again.")
            print("Falling back to template-based generation...")
            return generate_questions_with_templates(category, n)
        elif response.status_code == 503:
            print("🚧 Model Loading: The AI model is loading. This can take a few minutes.")
            print("Falling back to template-based generation for now...")
            return generate_questions_with_templates(category, n)
        else:
            print(f"❌ API Error (Status {response.status_code}): {response.text}")
            print("🔄 Falling back to template-based generation...")
            return generate_questions_with_templates(category, n)

    except requests.exceptions.RequestException as e:
        print(f"🌐 Connection Error: Unable to connect to Hugging Face API. {e}")
        print("🔄 Using template-based generation instead...")
        return generate_questions_with_templates(category, n)
    except Exception as e:
        print(f"❌ Unexpected Error: {e}")
        print("🔄 Using template-based generation instead...")
        return generate_questions_with_templates(category, n)


In [ ]:
# Test Hugging Face API Integration
print("🧪 Testing Hugging Face API Integration...")

# Load environment variables to access the API token
load_dotenv()
API_TOKEN = os.environ.get("HUGGINGFACE_API_TOKEN", "")

if API_TOKEN and API_TOKEN != "your_token_here":
    print("✅ API Token found - Testing connection...")

    # Test with a simple communication question generation
    test_df = generate_questions_with_huggingface('communication', n=3)

    if not test_df.empty and 'generated_by_huggingface' in test_df['source_url'].values:
        print(f"✅ AI Generation Success! Generated {len(test_df)} test questions:")
        for i, row in test_df.iterrows():
            print(f"{i+1}. {row['question_text']}")
    else:
        print("⚠️ AI generation had issues, but template fallback worked successfully!")
        print(f"📝 Generated {len(test_df)} template-based questions:")
        for i, row in test_df.iterrows():
            print(f"{i+1}. {row['question_text']}")

    print("\n💡 To enable AI generation, ensure your Hugging Face token has:")
    print(" 1. 'Read' permissions")
    print(" 2. 'Inference API' access")
    print(" 3. Create a new token at: https://huggingface.co/settings/tokens")
    print(" 4. Select 'Read' permission when creating the token")
else:
    print("⚠️ No API token found - using template-based generation only")
    print("To enable AI features, add your token to the .env file")

    # Test template-based generation instead
    test_df = generate_questions_with_templates('communication', n=3)
    print(f"✅ Template-based generation working! Generated {len(test_df)} test questions:")
    for i, row in test_df.iterrows():
        print(f"{i+1}. {row['question_text']}")

print("\n🎯 Assessment System Status:")
print("✅ Template-based generation: Working")
print("✅ Fallback mechanism: Working")
print("✅ Error handling: Working")
print("✅ Streamlit app: Ready to use")
print("\n" + "="*50)

🧪 Testing Hugging Face API Integration...
✅ API Token found - Testing connection...
Generating 3 questions for communication skills using Hugging Face...
🔐 Authentication Error: Invalid Hugging Face API token.
Please check your token in the .env file and try again.
Falling back to template-based generation...
Generating 3 questions for communication skills...
Successfully generated 3 questions for communication skills
⚠️ AI generation had issues, but template fallback worked successfully!
📝 Generated 3 template-based questions:
1. I can express ideas cross-functional teams without losing their attention.
2. I can express ideas stakeholders without losing their attention.
3. I negotiate clients clearly and concisely.

💡 To enable AI generation, ensure your Hugging Face token has:
 1. 'Read' permissions
 2. 'Inference API' access
 3. Create a new token at: https://huggingface.co/settings/tokens
 4. Select 'Read' permission when creating the token

🎯 Assessment System Status:
✅ Template-b

## 4.Question Templates and Components

These functions define the templates and components used for generating questions for each skill category.
Each category has specialized templates and vocabulary to ensure relevant, meaningful questions.

In [65]:
def get_question_templates(category):
    """Get templates for generating questions for a specific category"""
    templates = {
        'communication': [
            "I effectively {action} when {context}.",
            "I am skilled at {action} {modifier}.",
            "I {action} {audience} {modifier}.",
            "I find it easy to {action} even when {challenge}.",
            "When {situation}, I {action} {modifier}.",
            "I {action} {modifier} rather than {alternative_action}.",
            "My {communication_type} communication is {quality}.",
            "I {action} {medium} {modifier}.",
            "I can {action} {audience} without {negative_outcome}.",
            "During {meeting_type} meetings, I {action} {modifier}."
        ],
        'leadership': [
            "I {leadership_action} to {leadership_outcome}.",
            "I am effective at {leadership_skill} in {leadership_context}.",
            "When facing {leadership_challenge}, I {leadership_response}.",
            "My team members would say I {leadership_quality}.",
            "I {leadership_frequency} {leadership_action} to ensure {leadership_goal}.",
            "I can {leadership_action} even when {leadership_obstacle}.",
            "I {leadership_approach} team members who {team_situation}.",
            "When projects {project_status}, I {leadership_intervention}.",
            "I create an environment where {team_benefit}.",
            "My leadership style emphasizes {leadership_emphasis} while maintaining {leadership_balance}."
        ],
        'time_management': [
            "I {time_action} to {time_outcome}.",
            "I effectively {time_skill} when {time_context}.",
            "I {time_frequency} {time_practice} to maximize productivity.",
            "When dealing with {time_challenge}, I {time_strategy}.",
            "I maintain {time_quality} by {time_method}.",
            "I can {time_ability} without {negative_time_outcome}.",
            "My approach to {time_situation} involves {time_technique}.",
            "I {time_habit} at the {workday_period} of each workday.",
            "When faced with {deadline_scenario}, I {deadline_response}.",
            "I {consistently} avoid {time_waster} by {prevention_method}."
        ],
        'analytical': [
            "When analyzing {analysis_object}, I {analysis_action} {analysis_modifier}.",
            "I am skilled at {analysis_skill} to {analysis_purpose}.",
            "I {analysis_frequency} {analysis_practice} when solving problems.",
            "When presented with {analysis_input}, I {analysis_process} before {analysis_output}.",
            "I can effectively {analysis_method} to {analysis_goal}.",
            "My approach to {problem_type} problems involves {analytical_approach}.",
            "When data shows {data_pattern}, I typically {data_response}.",
            "I {evaluation_action} multiple {evaluation_subject} before {decision_action}.",
            "My {analytical_strength} helps me overcome {analytical_challenge}.",
            "I can {complex_action} complex information to {simplification_outcome}."
        ]
    }

    return templates.get(category, [])


## 5.Question Generation Utilities

These utilities handle the actual generation of questions, combining templates with components
and managing the flow between scraped and generated content.

In [66]:
def get_question_components(category):
    """Get components for filling templates for a specific category"""
    components = {
        'communication': {
            "action": [
                "communicate", "listen", "express ideas", "provide feedback", "ask questions",
                "articulate thoughts", "share information", "convey messages", "present data",
                "explain concepts", "respond to concerns", "clarify misunderstandings",
                "negotiate", "persuade", "mediate discussions", "facilitate conversations"
            ],
            "context": [
                "in team meetings", "with senior management", "with clients",
                "in high-pressure situations", "across departments", "with remote colleagues",
                "in cross-cultural settings", "during performance reviews", "in conflict situations"
            ],
            "modifier": [
                "clearly and concisely", "with confidence", "in a structured manner",
                "with empathy", "effectively", "in a timely manner", "tactfully",
                "with appropriate detail", "proactively", "in a persuasive way",
                "without creating confusion", "while maintaining attention"
            ],
            "audience": [
                "team members", "stakeholders", "clients", "management", "cross-functional teams",
                "technical staff", "non-technical audiences", "difficult personalities",
                "diverse groups", "remote teams"
            ],
            "challenge": [
                "under time constraints", "facing resistance", "dealing with complex topics",
                "in stressful situations", "working with limited information", "addressing conflicts",
                "speaking to large groups", "communicating bad news", "handling objections"
            ],
            "situation": [
                "in conflict situations", "during project discussions", "in performance reviews",
                "in team brainstorming", "during client presentations", "in negotiation scenarios",
                "when receiving criticism", "during status updates", "when explaining changes"
            ],
            "frequently": [
                "consistently", "regularly", "proactively", "habitually", "actively",
                "deliberately", "consciously", "thoroughly", "systematically"
            ],
            "alternative_action": [
                "assuming understanding", "avoiding difficult conversations", "using jargon",
                "speaking too quickly", "interrupting others", "dominating discussions",
                "avoiding eye contact", "sending lengthy emails", "diluting the message"
            ],
            "communication_type": [
                "written", "verbal", "nonverbal", "visual", "email", "presentation",
                "interpersonal", "group", "crisis", "technical", "cross-cultural"
            ],
            "quality": [
                "clear and effective", "concise and impactful", "well-structured",
                "engaging and persuasive", "audience-appropriate", "culturally sensitive",
                "free of unnecessary jargon", "logically organized", "emotionally intelligent"
            ],
            "medium": [
                "through email", "in presentations", "in written documentation", "in meetings",
                "via video conferences", "through visual aids", "in one-page summaries",
                "using storytelling", "with data visualizations", "in status reports"
            ],
            "negative_outcome": [
                "causing confusion", "creating resistance", "losing their attention",
                "overwhelming them with details", "using too much jargon", "being misunderstood",
                "being perceived as condescending", "creating unnecessary tension",
                "missing important feedback", "overlooking cultural sensitivities"
            ],
            "meeting_type": [
                "team", "client", "stakeholder", "project review", "brainstorming", "strategy",
                "board", "all-hands", "cross-functional", "one-on-one", "performance review"
            ]
        },
        'leadership': {
            "leadership_action": [
                "motivate team members", "set clear expectations", "delegate responsibilities",
                "provide constructive feedback", "recognize achievements", "develop talent",
                "empower others", "build consensus", "establish trust", "champion change"
            ],
            "leadership_outcome": [
                "achieve team goals", "improve team performance", "build a positive culture",
                "increase engagement", "drive innovation", "resolve conflicts",
                "enhance collaboration", "develop future leaders", "overcome obstacles"
            ],
            "leadership_skill": [
                "making difficult decisions", "coaching team members", "managing change",
                "strategic planning", "crisis management", "building consensus",
                "giving constructive feedback", "recognizing talent", "addressing conflicts"
            ],
            "leadership_context": [
                "challenging times", "periods of growth", "organizational changes",
                "cross-functional projects", "remote work environments",
                "high-pressure situations", "cultural transformations", "restructuring"
            ],
            "leadership_challenge": [
                "team conflicts", "performance issues", "resource constraints",
                "organizational changes", "competing priorities", "tight deadlines",
                "resistance to change", "skill gaps", "communication breakdowns"
            ],
            "leadership_response": [
                "seek input from all stakeholders", "make decisive judgments",
                "communicate transparently", "adapt my approach", "provide additional support",
                "lead by example", "facilitate collaboration", "remain calm and focused"
            ],
            "leadership_quality": [
                "inspire them to do their best work", "provide clear direction",
                "listen to their concerns", "trust them with important tasks",
                "support their professional development", "give honest feedback",
                "recognize their strengths", "help them overcome challenges"
            ],
            "leadership_frequency": [
                "consistently", "regularly", "proactively", "deliberately", "systematically"
            ],
            "leadership_goal": [
                "team success", "individual growth", "high-quality outcomes",
                "organizational alignment", "continuous improvement",
                "innovation", "employee satisfaction", "operational excellence"
            ],
            "leadership_obstacle": [
                "facing resistance", "dealing with limited resources",
                "under tight deadlines", "during organizational change",
                "managing conflicting priorities", "addressing poor performance"
            ],
            "leadership_approach": [
                "mentor", "coach", "guide", "support", "challenge",
                "motivate", "empower", "direct"
            ],
            "team_situation": [
                "are underperforming", "show exceptional talent",
                "face personal challenges", "disagree with team direction",
                "need development", "demonstrate initiative"
            ],
            "project_status": [
                "fall behind schedule", "exceed expectations",
                "face unexpected obstacles", "require scope changes",
                "reveal team conflicts", "need additional resources"
            ],
            "leadership_intervention": [
                "reassess priorities", "reallocate resources",
                "provide additional guidance", "facilitate problem-solving sessions",
                "communicate changes clearly", "acknowledge team efforts"
            ],
            "team_benefit": [
                "innovation is rewarded", "mistakes are viewed as learning opportunities",
                "diverse perspectives are valued", "collaboration is the norm",
                "individual strengths are leveraged", "continuous learning is encouraged"
            ],
            "leadership_emphasis": [
                "results", "people development", "innovation",
                "process improvement", "strategic thinking", "relationship building"
            ],
            "leadership_balance": [
                "accountability", "work-life balance", "individual autonomy",
                "team cohesion", "attention to detail", "big-picture thinking"
            ]
        },
        'time_management': {
            "time_action": [
                "prioritize tasks", "create schedules", "set clear deadlines",
                "eliminate distractions", "delegate effectively", "chunk similar activities",
                "use time-blocking techniques", "maintain to-do lists", "track time usage"
            ],
            "time_outcome": [
                "meet deadlines consistently", "maximize productivity", "reduce stress",
                "balance multiple responsibilities", "achieve work-life balance",
                "increase focus time", "improve work quality", "create buffer for emergencies"
            ],
            "time_skill": [
                "managing multiple deadlines", "planning my workday", "estimating task duration",
                "tracking my time usage", "adjusting priorities", "saying no when necessary",
                "recognizing time-wasting activities", "maintaining focus", "batching similar tasks"
            ],
            "time_context": [
                "working under pressure", "handling multiple projects", "faced with interruptions",
                "deadlines change", "new tasks are assigned", "priorities shift",
                "unexpected issues arise", "collaborating across time zones", "during peak workloads"
            ],
            "time_frequency": [
                "consistently", "regularly", "daily", "at the start of each week", "proactively",
                "at the end of each day", "between major tasks", "during low-energy periods"
            ],
            "time_practice": [
                "use to-do lists", "break large tasks into smaller steps", "set specific goals",
                "block time for focused work", "review progress regularly",
                "eliminate low-value activities", "batch similar tasks", "schedule buffer time"
            ],
            "time_challenge": [
                "unexpected interruptions", "shifting priorities", "tight deadlines",
                "multiple competing tasks", "complex projects", "email overload",
                "meeting-heavy days", "scope creep", "procrastination tendencies"
            ],
            "time_strategy": [
                "reassess priorities", "communicate timeline changes", "find efficient shortcuts",
                "seek additional resources", "eliminate non-essential tasks",
                "delegate appropriate tasks", "extend deadlines when necessary", "work in focused sprints"
            ],
            "time_quality": [
                "a well-organized schedule", "clear priorities", "focus during work hours",
                "reasonable workload", "effective time allocation", "protected focus time",
                "work-life boundaries", "energy for important tasks", "buffer for emergencies"
            ],
            "time_method": [
                "planning ahead", "using productivity tools", "setting boundaries",
                "regularly reviewing commitments", "avoiding multitasking",
                "implementing time-boxing", "creating routines", "tracking time usage"
            ],
            "time_ability": [
                "meet tight deadlines", "handle multiple priorities", "stay focused for extended periods",
                "estimate task durations accurately", "adapt to changing schedules",
                "identify time-wasting activities", "complete important tasks first"
            ],
            "negative_time_outcome": [
                "becoming overwhelmed", "sacrificing quality", "working excessive hours",
                "missing important details", "feeling stressed", "neglecting self-care",
                "delaying important decisions", "creating bottlenecks for others"
            ],
            "time_situation": [
                "busy periods", "multiple deadlines", "long-term projects",
                "unexpected work", "recurring tasks", "meetings and interruptions",
                "email management", "decision-making", "planning processes"
            ],
            "time_technique": [
                "prioritization matrices", "time-blocking", "the Pomodoro technique",
                "delegation", "saying no to low-value requests", "batching similar tasks",
                "using templates for recurring work", "setting clear boundaries"
            ],
            "time_habit": [
                "plan my priorities", "review my calendar", "check my progress",
                "clear my inbox", "update my to-do list", "reflect on accomplishments",
                "prepare for upcoming tasks", "eliminate distractions"
            ],
            "workday_period": [
                "beginning", "end", "most productive hours", "before meetings",
                "after lunch", "between focused work sessions", "during commute time"
            ],
            "deadline_scenario": [
                "multiple simultaneous deadlines", "unexpected urgent requests",
                "shortened timelines", "scope increases without timeline changes",
                "dependent tasks delayed by others", "resource limitations"
            ],
            "deadline_response": [
                "renegotiate timelines when appropriate", "focus on the most critical deliverables first",
                "seek additional resources", "adjust my work schedule temporarily",
                "communicate progress transparently", "simplify deliverables when possible"
            ],
            "consistently": [
                "proactively", "routinely", "methodically", "deliberately", "systematically"
            ],
            "time_waster": [
                "unnecessary meetings", "constant email checking", "multitasking",
                "social media distractions", "perfectionism", "unclear priorities",
                "disorganized workspaces", "unproductive conversations"
            ],
            "prevention_method": [
                "establishing clear boundaries", "implementing specific routines",
                "using productivity tools", "blocking distracting websites",
                "setting clear agendas for meetings", "batching similar activities"
            ]
        },
        'analytical': {
            "analysis_object": [
                "data", "problems", "complex situations", "project requirements",
                "market trends", "customer feedback", "performance metrics",
                "research findings", "competitive information", "process inefficiencies"
            ],
            "analysis_action": [
                "identify patterns", "draw logical conclusions", "evaluate options",
                "determine root causes", "make evidence-based decisions",
                "spot anomalies", "recognize relationships", "quantify impacts"
            ],
            "analysis_modifier": [
                "systematically", "objectively", "thoroughly", "efficiently",
                "with attention to detail", "considering multiple perspectives",
                "without bias", "using established frameworks", "holistically"
            ],
            "analysis_skill": [
                "breaking down complex problems", "interpreting data", "identifying connections",
                "evaluating evidence", "distinguishing facts from assumptions",
                "recognizing patterns", "quantifying variables", "testing hypotheses"
            ],
            "analysis_purpose": [
                "find optimal solutions", "make informed decisions", "identify improvement opportunities",
                "predict outcomes", "mitigate risks", "validate hypotheses",
                "understand root causes", "establish benchmarks", "develop strategies"
            ],
            "analysis_frequency": [
                "consistently", "methodically", "routinely", "deliberately", "systematically"
            ],
            "analysis_practice": [
                "gather all relevant information", "consider alternative explanations",
                "test assumptions", "evaluate the reliability of sources",
                "separate facts from opinions", "use structured problem-solving methods",
                "document my reasoning", "seek disconfirming evidence"
            ],
            "analysis_input": [
                "conflicting information", "incomplete data", "complex problems",
                "ambiguous requirements", "multiple variables", "uncertain conditions",
                "stakeholder disagreements", "contradictory evidence", "legacy assumptions"
            ],
            "analysis_process": [
                "identify key factors", "evaluate different perspectives", "apply logical frameworks",
                "test multiple hypotheses", "prioritize critical information",
                "map interdependencies", "calculate probabilities", "validate data quality"
            ],
            "analysis_output": [
                "making recommendations", "drawing conclusions", "implementing solutions",
                "communicating findings", "making decisions", "creating action plans",
                "developing predictive models", "establishing measurement criteria"
            ],
            "analysis_method": [
                "use data visualization", "apply statistical methods", "conduct root cause analysis",
                "create decision matrices", "perform scenario analysis", "develop comparative frameworks",
                "construct logic models", "implement structured evaluation methods"
            ],
            "analysis_goal": [
                "solve complex problems", "identify improvement opportunities", "optimize processes",
                "make data-driven decisions", "predict future trends", "mitigate risks",
                "validate hypotheses", "eliminate inefficiencies", "support strategic goals"
            ],
            "problem_type": [
                "data-intensive", "multi-variable", "ambiguous", "technical",
                "resource allocation", "process optimization", "strategic",
                "time-sensitive", "interdependent"
            ],
            "analytical_approach": [
                "breaking the problem into components", "identifying underlying patterns",
                "quantifying variables when possible", "applying structured frameworks",
                "using both deductive and inductive reasoning", "testing multiple hypotheses"
            ],
            "data_pattern": [
                "unexpected anomalies", "conflicting trends", "statistical outliers",
                "correlation between variables", "cyclical patterns", "significant gaps",
                "skewed distributions", "contradictory indicators"
            ],
            "data_response": [
                "investigate potential causes", "validate data accuracy first",
                "perform additional analysis", "consider alternative interpretations",
                "consult subject matter experts", "look for contextual factors"
            ],
            "evaluation_action": [
                "compare", "assess", "weigh", "measure", "test", "validate"
            ],
            "evaluation_subject": [
                "options", "approaches", "hypotheses", "data sources",
                "interpretations", "methodologies", "assumptions"
            ],
            "decision_action": [
                "drawing conclusions", "making recommendations", "finalizing a course of action",
                "committing resources", "implementing solutions", "communicating results"
            ],
            "analytical_strength": [
                "attention to detail", "pattern recognition", "logical reasoning",
                "quantitative analysis", "critical thinking", "systems perspective",
                "objectivity", "methodical approach"
            ],
            "analytical_challenge": [
                "information overload", "ambiguous problems", "tight deadlines",
                "confirmation bias", "incomplete data", "complex interdependencies",
                "changing requirements", "qualitative variables"
            ],
            "complex_action": [
                "distill", "synthesize", "translate", "organize", "structure", "visualize"
            ],
            "simplification_outcome": [
                "communicate key insights", "facilitate decision making",
                "enable stakeholder understanding", "identify action priorities",
                "clarify complex relationships", "highlight critical factors"
            ]
        }
    }

    return components.get(category, {})


def fill_template_with_components(template, components):
    """Fill a template with randomly selected components"""
    filled_template = template

    # Find all the placeholders in this template
    placeholders = re.findall(r'{([^}]+)}', template)

    # Replace each placeholder with a random choice from the corresponding component list
    for placeholder in placeholders:
        if placeholder in components:
            replacement = random.choice(components[placeholder])
            filled_template = filled_template.replace(f"{{{placeholder}}}", replacement)

    return filled_template


def get_soft_skills_questions(category, count_needed=100, use_llm=False):
    """
    Get soft skills questions through both scraping and generation

    Args:
        category (str): One of 'communication', 'leadership', 'time_management', 'analytical'
        count_needed (int): Total number of questions needed
        use_llm (bool): Whether to use Hugging Face LLM (True) or templates (False)

    Returns:
        pd.DataFrame: DataFrame containing questions from both sources
    """
    print(f"Getting {count_needed} questions for {category} skills...")

    # First try to get questions through scraping
    scraped_df = scrape_soft_skills_questions(category)

    # Calculate how many more questions we need after scraping
    scraped_count = len(scraped_df)
    generated_count_needed = max(0, count_needed - scraped_count)

    # If we need more questions, generate them
    if generated_count_needed > 0:
        print(f"Need {generated_count_needed} more questions. Generating them...")

        if use_llm:
            generated_df = generate_questions_with_huggingface(category, n=generated_count_needed)
        else:
            generated_df = generate_questions_with_templates(category, n=generated_count_needed)

        # Combine scraped and generated questions
        if not scraped_df.empty:
            combined_df = pd.concat([scraped_df, generated_df], ignore_index=True)
        else:
            combined_df = generated_df
    else:
        print("Enough questions were scraped. No need for generation.")
        combined_df = scraped_df

    # Ensure we have exactly count_needed questions (or as close as possible)
    if len(combined_df) > count_needed:
        combined_df = combined_df.sample(count_needed, random_state=42).reset_index(drop=True)

    print(f"Final question count for {category}: {len(combined_df)}")
    return combined_df





## 6.Interactive Assessment UI

This section creates an interactive HTML-based assessment form that allows users to
respond to questions on a 5-point scale and receive immediate feedback on their skills.

In [67]:
# Function to create an interactive assessment interface
def create_assessment(questions_df, category):
    """Create an HTML assessment form for the given category and questions"""

    # Create a random ID for this assessment
    assessment_id = f"{category}_{random.randint(1000, 9999)}"

    # Start building the HTML form
    html = f"""
    <div class="assessment-container" style="max-width: 800px; margin: 0 auto; font-family: Arial, sans-serif;">
        <h2 style="color: #000; text-align: center;">{category.replace('_', ' ').title()} Skills Assessment</h2>
        <p style="color: #000;">Rate yourself on each statement from 1 (Strongly Disagree) to 5 (Strongly Agree).</p>
        <form id="{assessment_id}">
            <table style="width: 100%; border-collapse: collapse;">
                <tr style="background-color: #e6f2ff;">
                    <th style="padding: 12px; text-align: left; border-bottom: 1px solid #ddd; color: #000;">Statement</th>
                    <th style="padding: 12px; text-align: center; border-bottom: 1px solid #ddd; color: #000;">1<br><small style="color: #000;">Strongly<br>Disagree</small></th>
                    <th style="padding: 12px; text-align: center; border-bottom: 1px solid #ddd; color: #000;">2<br><small style="color: #000;">Disagree</small></th>
                    <th style="padding: 12px; text-align: center; border-bottom: 1px solid #ddd; color: #000;">3<br><small style="color: #000;">Neutral</small></th>
                    <th style="padding: 12px; text-align: center; border-bottom: 1px solid #ddd; color: #000;">4<br><small style="color: #000;">Agree</small></th>
                    <th style="padding: 12px; text-align: center; border-bottom: 1px solid #ddd; color: #000;">5<br><small style="color: #000;">Strongly<br>Agree</small></th>
                </tr>
    """

    # Add each question as a row in the table
    for i, row in questions_df.iterrows():
        question_id = f"q_{i}"
        question_text = row['question_text']

        html += f"""
        <tr style="border-bottom: 1px solid #ddd;">
            <td style="padding: 12px;">{question_text}</td>
        """

        # Add radio buttons for each rating option
        for rating in range(1, 6):
            html += f"""
            <td style="text-align: center;">
                <input type="radio" name="{question_id}" value="{rating}" required>
            </td>
            """

        html += "</tr>"

    # Add submit button and closing tags
    html += f"""
            </table>
            <div style="margin-top: 40px; margin-bottom: 60px; text-align: center; background-color: white; padding: 15px; border-radius: 8px;">
                <button type="button" id="submit-btn-{assessment_id}" style="background-color: #4CAF50; color: white; padding: 15px 30px; border: 2px solid black; border-radius: 4px; cursor: pointer; font-size: 18px; font-weight: bold; box-shadow: 0 4px 8px rgba(0,0,0,0.2);">SUBMIT ASSESSMENT</button>
            </div>
        </form>
        <div id="results-{assessment_id}" style="margin-top: 20px; display: none;">
            <h3>Your Results</h3>
            <div id="score-{assessment_id}"></div>
            <div id="feedback-{assessment_id}"></div>
        </div>
        <script>
        document.getElementById("submit-btn-{assessment_id}").addEventListener("click", function() {{
            // Simple scoring logic
            let form = document.getElementById("{assessment_id}");
            let total = 0;
            let answered = 0;
            let questions = {len(questions_df)};

            for (let i = 0; i < questions; i++) {{
                let name = "q_" + i;
                let selected = document.querySelector('input[name="' + name + '"]:checked');
                if (selected) {{
                    total += parseInt(selected.value);
                    answered++;
                }}
            }}

            if (answered < questions) {{
                alert("Please answer all questions before submitting.");
                return;
            }}

            let average = total / questions;
            let percentage = (average / 5) * 100;

            // Display results
            document.getElementById("score-{assessment_id}").innerHTML = '<p>Your average score: <strong>' + average.toFixed(1) + '/5</strong> (' + percentage.toFixed(1) + '%)</p>';

            // Generate feedback based on score
            let feedback = '';
            if (average >= 4.5) {{
                feedback = 'Outstanding! You demonstrate excellent {category.replace('_', ' ')} skills.';
            }} else if (average >= 3.5) {{
                feedback = 'Good job! You have solid {category.replace('_', ' ')} skills with some room for improvement.';
            }} else if (average >= 2.5) {{
                feedback = 'You have moderate {category.replace('_', ' ')} skills. Consider focusing on development in this area.';
            }} else {{
                feedback = 'This appears to be an area for growth. Consider seeking resources to develop your {category.replace('_', ' ')} skills.';
            }}

            document.getElementById("feedback-{assessment_id}").innerHTML = '<p>' + feedback + '</p>';
            document.getElementById("results-{assessment_id}").style.display = "block";
        }});
        </script>
    </div>
    """

    return HTML(html)

# Main execution section
if __name__ == "__main__":
    # Get questions for all categories
    all_questions = {}
    for category in ['communication', 'leadership', 'time_management', 'analytical']:
        print(f"\n--- Processing {category} skills questions ---\n")

        use_llm = True

        # Get questions for this category
        all_questions[category] = get_soft_skills_questions(
            category=category,
            count_needed=50, # Adjust as needed
            use_llm=use_llm
        )

    # Save all questions to CSV files
    for category, df in all_questions.items():
        file_path = f"./data/{category}_questions.csv"
        df.to_csv(file_path, index=False)
        print(f"Saved {len(df)} {category} questions to {file_path}")

    # Save combined questions
    combined_df = pd.concat([df for df in all_questions.values()], ignore_index=True)
    combined_df.to_csv("./data/all_soft_skills_questions.csv", index=False)
    print(f"Saved total of {len(combined_df)} questions to ./data/all_soft_skills_questions.csv")




--- Processing communication skills questions ---

Getting 50 questions for communication skills...
Scraping 7 questions from https://hr-survey.com/360_Survey_Example_1.htm...
Found question: Takes on challenging questions and provides instan...
Found question: Coaches others on their written communication skil...
Found question: Addresses issues of key importance to stakeholders...
Found question: Communicates goals of project, resources required,...
Found question: Articulates ideas and emotions clearly to others....
Found question: Gives clear and convincing presentations....
Scraping 4 questions from https://hr-survey.com/360Feedback_Sample_Assessment_Form.htm...
Found question: Adapts language and terminology to meet the needs ...
Found question: Recaps action steps from meetings to ensure clarit...
Found question: Considers the audience in how the communication is...
Found question: Responds in a timely manner, respecting deadlines ...
Total communication questions scraped: 10
N

## 7.Main Execution

This section runs the entire assessment generation pipeline:
1. Collects questions for each skill category through scraping and generation
2. Saves the questions to CSV files for future use
3. Creates a mixed assessment with questions from all categories
4. Displays the assessment for immediate use

In [68]:
# Demo: Create and display a mixed assessment with questions from all categories
print("\nCreating a mixed soft skills assessment...")

# Sample questions from each category
mixed_questions = pd.DataFrame()
questions_per_category = 5 # Number of questions to include from each category

for category in ['communication', 'leadership', 'time_management', 'analytical']:
    # Get a sample of questions from this category
    if category in all_questions and not all_questions[category].empty:
        category_questions = all_questions[category].sample(min(questions_per_category, len(all_questions[category])))

        # Add to our mixed questions dataframe
        mixed_questions = pd.concat([mixed_questions, category_questions], ignore_index=True)

# Shuffle the questions to mix up the categories
mixed_questions = mixed_questions.sample(frac=1).reset_index(drop=True)

# Display information about the mixed assessment
print(f"Created mixed assessment with {len(mixed_questions)} total questions:")
for category in ['communication', 'leadership', 'time_management', 'analytical']:
    count = len(mixed_questions[mixed_questions['category'] == category])
    print(f"- {category}: {count} questions")

# Create and display the mixed assessment
display(create_assessment(mixed_questions, "comprehensive_soft_skills"))


Creating a mixed soft skills assessment...
Created mixed assessment with 20 total questions:
- communication: 5 questions
- leadership: 5 questions
- time_management: 5 questions
- analytical: 5 questions


Statement,1StronglyDisagree,2Disagree,3Neutral,4Agree,5StronglyAgree
I set clear expectations to enhance collaboration.,,,,,
I effectively saying no when necessary when during peak workloads.,,,,,
I recognize achievements to enhance collaboration.,,,,,
I systematically avoid unproductive conversations by batching similar activities.,,,,,
I effectively tracking my time usage when new tasks are assigned.,,,,,
"Communicates goals of project, resources required, resources available, etc. to the team",,,,,
My approach to strategic problems involves quantifying variables when possible.,,,,,
I deliberately separate facts from opinions when solving problems.,,,,,
I facilitate conversations using storytelling clearly and concisely.,,,,,
I effectively ask questions when during performance reviews.,,,,,


## 8. Clustering Analysis for Heterogeneous Student Group Formation

**Project Goal**: Use clustering algorithms to automatically create diverse and balanced student groups for collaborative learning.

### What is Clustering?
Clustering is an unsupervised machine learning technique that groups similar data points together. For student group formation:

1. **Student Profiling**: Identify students with similar skill patterns, learning styles, and characteristics
2. **Heterogeneous Group Creation**: Form diverse groups by selecting students from different clusters
3. **Balanced Team Composition**: Ensure each group has complementary skills and perspectives

### Applications in Educational Settings:
- **Collaborative Learning**: Create balanced teams for projects and assignments
- **Peer Learning**: Match students with complementary strengths and weaknesses  
- **Skill Development**: Expose students to diverse perspectives and approaches
- **Bias Reduction**: Remove instructor bias from manual group formation
- **Improved Outcomes**: Research shows heterogeneous groups enhance learning and critical thinking

### Clustering Process for Group Formation:
1. **Data Collection**: Gather student data (moyenne, skills, learning styles, interests, specializations)
2. **Clustering**: Group students with similar profiles using ML algorithms
3. **Group Formation**: Select one student from each cluster to create diverse teams
4. **Validation**: Ensure groups are balanced across key attributes

In [69]:
# Import additional libraries for clustering analysis
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

def generate_student_assessment_data(n_students=120, random_state=42):
    """
    Generate sample Tunisian IT engineering student assessment data for clustering and group formation
    
    Args:
        n_students (int): Number of sample students to generate
        random_state (int): Random seed for reproducibility
    
    Returns:
        pd.DataFrame: Sample student assessment responses with additional attributes
    """
    np.random.seed(random_state)
    
    # Define student archetypes based on learning styles and skill combinations
    # Adapted for Tunisian IT engineering students with moyenne (0-20 scale)
    student_archetypes = {
        'Technical Leader': {
            'communication': (3.5, 0.8),
            'leadership': (4.2, 0.6), 
            'time_management': (4.0, 0.7),
            'analytical': (4.5, 0.5),
            'moyenne_range': (14.0, 18.0),  # Good to excellent students (70-90%)
            'learning_styles': ['Visual', 'Logical'],
            'interests': ['Software Development', 'AI/Machine Learning', 'System Architecture']
        },
        'Collaborative Communicator': {
            'communication': (4.5, 0.5),
            'leadership': (3.8, 0.7),
            'time_management': (3.2, 0.9),
            'analytical': (2.8, 0.8),
            'moyenne_range': (12.0, 16.0),  # Average to good students (60-80%)
            'learning_styles': ['Auditory', 'Social'],
            'interests': ['Web Development', 'UI/UX Design', 'Project Management']
        },
        'Analytical Thinker': {
            'communication': (3.0, 0.8),
            'leadership': (2.5, 0.9),
            'time_management': (3.8, 0.7),
            'analytical': (4.6, 0.4),
            'moyenne_range': (15.0, 19.0),  # Very good to excellent students (75-95%)
            'learning_styles': ['Visual', 'Solitary'],
            'interests': ['Data Science', 'Algorithms', 'Research & Development']
        },
        'Well-Rounded Student': {
            'communication': (3.8, 0.6),
            'leadership': (3.5, 0.7),
            'time_management': (3.7, 0.6),
            'analytical': (3.6, 0.7),
            'moyenne_range': (13.0, 16.5),  # Good students (65-82.5%)
            'learning_styles': ['Kinesthetic', 'Visual'],
            'interests': ['Mobile Development', 'Network Security', 'Technology Innovation']
        },
        'Creative Innovator': {
            'communication': (4.0, 0.7),
            'leadership': (3.2, 0.8),
            'time_management': (2.8, 1.0),
            'analytical': (3.2, 0.9),
            'moyenne_range': (11.0, 15.0),  # Average students (55-75%)
            'learning_styles': ['Kinesthetic', 'Visual'],
            'interests': ['Game Development', 'Creative Coding', 'Startup Innovation']
        },
        'Emerging Scholar': {
            'communication': (2.8, 0.9),
            'leadership': (2.2, 0.8),
            'time_management': (2.5, 1.0),
            'analytical': (3.0, 0.9),
            'moyenne_range': (10.0, 13.5),  # Developing students (50-67.5%)
            'learning_styles': ['Auditory', 'Reading'],
            'interests': ['Programming Fundamentals', 'Database Systems', 'Self-Improvement']
        }
    }
    
    # IT engineering specializations for diversity (instead of different majors)
    specializations = [
        'Software Engineering', 'Data Science & AI', 'Cybersecurity', 
        'Network Engineering', 'Mobile Development', 'Web Development',
        'DevOps & Cloud Computing', 'Embedded Systems'
    ]
    
    # Programming languages for additional diversity
    programming_languages = [
        'Python', 'Java', 'JavaScript', 'C++', 'C#', 'PHP', 'React', 'Angular'
    ]
    
    learning_styles = ['Visual', 'Auditory', 'Kinesthetic', 'Reading', 'Social', 'Solitary', 'Logical']
    
    sample_data = []
    archetype_names = list(student_archetypes.keys())
    
    for i in range(n_students):
        # Randomly assign archetype (with some mixing for realism)
        if np.random.random() < 0.75:  # 75% pure archetype
            archetype = np.random.choice(archetype_names)
            arch_data = student_archetypes[archetype]
        else:  # 25% mixed profiles
            arch1, arch2 = np.random.choice(archetype_names, 2, replace=False)
            # Blend two archetypes
            arch_data = {}
            for skill in ['communication', 'leadership', 'time_management', 'analytical']:
                mean1, std1 = student_archetypes[arch1][skill]
                mean2, std2 = student_archetypes[arch2][skill]
                arch_data[skill] = ((mean1 + mean2) / 2, (std1 + std2) / 2)
            
            # Mix other attributes
            moyenne1_min, moyenne1_max = student_archetypes[arch1]['moyenne_range']
            moyenne2_min, moyenne2_max = student_archetypes[arch2]['moyenne_range']
            arch_data['moyenne_range'] = ((moyenne1_min + moyenne2_min) / 2, (moyenne1_max + moyenne2_max) / 2)
            arch_data['learning_styles'] = student_archetypes[arch1]['learning_styles'] + student_archetypes[arch2]['learning_styles']
            arch_data['interests'] = student_archetypes[arch1]['interests'] + student_archetypes[arch2]['interests']
            archetype = f"Mixed Profile"
        
        # Generate student data
        student_data = {
            'student_id': f"ETU_{i+1:03d}",  # ETU for Étudiant (French for student)
            'archetype': archetype
        }
        
        # Generate soft skill scores
        for skill in ['communication', 'leadership', 'time_management', 'analytical']:
            mean, std = arch_data[skill]
            score = np.random.normal(mean, std)
            score = max(1.0, min(5.0, score))
            student_data[f'{skill}_score'] = round(score, 2)
        
        # Generate Moyenne (0-20 scale instead of GPA)
        moyenne_min, moyenne_max = arch_data['moyenne_range']
        moyenne = np.random.uniform(moyenne_min, moyenne_max)
        student_data['moyenne'] = round(moyenne, 2)
        
        # Assign IT engineering specific attributes
        student_data['specialization'] = np.random.choice(specializations)
        student_data['preferred_programming_language'] = np.random.choice(programming_languages)
        student_data['primary_learning_style'] = np.random.choice(arch_data['learning_styles'])
        student_data['secondary_learning_style'] = np.random.choice(learning_styles)
        student_data['primary_interest'] = np.random.choice(arch_data['interests'])
        
        # Add some random diversity
        student_data['previous_group_experience'] = np.random.choice(['Faible', 'Moyen', 'Élevé'])  # French: Low, Medium, High
        student_data['extroversion_score'] = round(np.random.uniform(1, 5), 2)
        student_data['creativity_score'] = round(np.random.uniform(1, 5), 2)
        student_data['technical_confidence'] = round(np.random.uniform(1, 5), 2)  # New attribute for IT students
        
        sample_data.append(student_data)
    
    return pd.DataFrame(sample_data)

# Generate sample student data for clustering demonstration
print("👥 Generating sample Tunisian IT engineering student assessment data for heterogeneous group formation...")
students_df = generate_student_assessment_data(n_students=120)

# Display basic statistics
print(f"Generated data for {len(students_df)} IT engineering students")
print(f"Skill categories: {[col for col in students_df.columns if col.endswith('_score')]}")
print(f"Student archetypes: {students_df['archetype'].value_counts().to_dict()}")
print(f"Specializations: {students_df['specialization'].value_counts().to_dict()}")
print("\nSample student data preview:")
display(students_df.head(10))

# Display summary statistics
print("\n📊 Summary Statistics:")
skill_cols = [col for col in students_df.columns if col.endswith('_score')]
display(students_df[skill_cols + ['moyenne']].describe())

👥 Generating sample Tunisian IT engineering student assessment data for heterogeneous group formation...
Generated data for 120 IT engineering students
Skill categories: ['communication_score', 'leadership_score', 'time_management_score', 'analytical_score', 'extroversion_score', 'creativity_score']
Student archetypes: {'Mixed Profile': 26, 'Collaborative Communicator': 19, 'Analytical Thinker': 17, 'Technical Leader': 16, 'Creative Innovator': 15, 'Emerging Scholar': 15, 'Well-Rounded Student': 12}
Specializations: {'DevOps & Cloud Computing': 22, 'Cybersecurity': 19, 'Embedded Systems': 17, 'Software Engineering': 16, 'Mobile Development': 12, 'Web Development': 12, 'Network Engineering': 12, 'Data Science & AI': 10}

Sample student data preview:


,student_id,archetype,communication_score,leadership_score,time_management_score,analytical_score,moyenne,specialization,preferred_programming_language,primary_learning_style,secondary_learning_style,primary_interest,previous_group_experience,extroversion_score,creativity_score,technical_confidence
0,ETU_001,Creative Innovator,4.38,2.71,1.60,5.00,11.40,Cybersecurity,Angular,Kinesthetic,Reading,Startup Innovation,Moyen,1.23,3.89,4.75
1,ETU_002,Well-Rounded Student,3.46,2.85,2.13,4.27,14.02,Cybersecurity,React,Visual,Reading,Technology Innovation,Moyen,1.36,3.47,2.53
2,ETU_003,Mixed Profile,3.82,4.31,3.67,2.72,14.15,DevOps & Cloud Computing,C++,Auditory,Logical,Project Management,Moyen,2.56,1.73,4.02
3,ETU_004,Emerging Scholar,1.60,2.36,3.24,3.15,10.31,DevOps & Cloud Computing,C++,Auditory,Solitary,Programming Fundamentals,Faible,2.09,4.31,2.43
4,ETU_005,Well-Rounded Student,3.23,2.77,2.57,2.64,14.03,DevOps & Cloud Computing,JavaScript,Kinesthetic,Visual,Technology Innovation,Élevé,4.09,1.30,2.43
5,ETU_006,Technical Leader,2.29,4.86,3.88,4.29,15.30,Cybersecurity,C++,Logical,Solitary,System Architecture,Élevé,2.89,1.48,3.85
6,ETU_007,Mixed Profile,3.19,2.13,3.22,3.21,13.16,Mobile Development,Angular,Visual,Solitary,Network Security,Élevé,3.42,3.16,1.81
7,ETU_008,Mixed Profile,4.13,4.51,3.62,4.07,13.25,Web Development,PHP,Kinesthetic,Auditory,Game Development,Faible,2.27,1.44,1.91
8,ETU_009,Well-Rounded Student,2.99,2.79,3.78,2.73,14.70,Software Engineering,JavaScript,Visual,Reading,Technology Innovation,Élevé,2.29,3.08,3.81
9,ETU_010,Technical Leader,2.82,3.71,4.76,5.00,14.59,Embedded Systems,Python,Visual,Visual,AI/Machine Learning,Moyen,2.64,1.13,2.38



📊 Summary Statistics:


,communication_score,leadership_score,time_management_score,analytical_score,extroversion_score,creativity_score,moyenne
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,3.580833,3.266500,3.376583,3.708583,2.982083,2.727167,14.242500
std,0.855177,0.915316,0.930456,0.918305,1.097349,1.162599,1.798018
min,1.510000,1.150000,1.000000,1.470000,1.070000,1.000000,10.090000
25%,3.007500,2.642500,2.870000,3.087500,2.105000,1.632500,13.002500
50%,3.580000,3.330000,3.520000,3.765000,3.035000,2.785000,14.505000
75%,4.302500,3.902500,3.987500,4.467500,3.862500,3.665000,15.392500
max,5.000000,5.000000,5.000000,5.000000,5.000000,4.940000,18.650000


In [70]:
def prepare_clustering_data(df):
    """
    Prepare Tunisian IT engineering student data for clustering by encoding and scaling features
    
    Args:
        df (pd.DataFrame): Student assessment data
    
    Returns:
        tuple: (scaled_features, feature_names, label_encoders)
    """
    # Select features for clustering
    skill_features = [col for col in df.columns if col.endswith('_score')]
    other_features = ['moyenne', 'extroversion_score', 'creativity_score', 'technical_confidence']
    categorical_features = ['specialization', 'preferred_programming_language', 'primary_learning_style', 'previous_group_experience']
    
    # Prepare numerical features
    numerical_data = df[skill_features + other_features].copy()
    
    # Encode categorical features
    label_encoders = {}
    categorical_data = pd.DataFrame()
    
    for feature in categorical_features:
        le = LabelEncoder()
        categorical_data[feature] = le.fit_transform(df[feature])
        label_encoders[feature] = le
    
    # Combine all features
    combined_features = pd.concat([numerical_data, categorical_data], axis=1)
    
    # Scale features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(combined_features)
    
    feature_names = list(combined_features.columns)
    
    return scaled_features, feature_names, label_encoders, scaler

def find_optimal_clusters(data, max_clusters=10):
    """
    Find optimal number of clusters using elbow method and silhouette analysis
    
    Args:
        data: Scaled feature data
        max_clusters (int): Maximum number of clusters to test
    
    Returns:
        dict: Results with optimal cluster numbers
    """
    inertias = []
    silhouette_scores = []
    calinski_scores = []
    cluster_range = range(2, max_clusters + 1)
    
    for n_clusters in cluster_range:
        # K-Means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(data)
        
        # Calculate metrics
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(data, cluster_labels))
        calinski_scores.append(calinski_harabasz_score(data, cluster_labels))
    
    # Find optimal number using silhouette score
    optimal_silhouette = cluster_range[np.argmax(silhouette_scores)]
    
    return {
        'cluster_range': list(cluster_range),
        'inertias': inertias,
        'silhouette_scores': silhouette_scores,
        'calinski_scores': calinski_scores,
        'optimal_clusters': optimal_silhouette
    }

def perform_clustering(data, n_clusters, algorithm='kmeans'):
    """
    Perform clustering using specified algorithm
    
    Args:
        data: Scaled feature data
        n_clusters (int): Number of clusters
        algorithm (str): Clustering algorithm ('kmeans', 'hierarchical', 'dbscan')
    
    Returns:
        array: Cluster labels
    """
    if algorithm == 'kmeans':
        clusterer = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    elif algorithm == 'hierarchical':
        clusterer = AgglomerativeClustering(n_clusters=n_clusters)
    elif algorithm == 'dbscan':
        clusterer = DBSCAN(eps=0.5, min_samples=5)
    else:
        raise ValueError("Algorithm must be 'kmeans', 'hierarchical', or 'dbscan'")
    
    cluster_labels = clusterer.fit_predict(data)
    return cluster_labels

# Prepare data for clustering
print("🔄 Preparing data for clustering analysis...")
scaled_features, feature_names, label_encoders, scaler = prepare_clustering_data(students_df)

print(f"Features used for clustering: {feature_names}")
print(f"Data shape: {scaled_features.shape}")

# Find optimal number of clusters
print("\n🔍 Finding optimal number of clusters...")
cluster_analysis = find_optimal_clusters(scaled_features, max_clusters=8)

print(f"Optimal number of clusters (silhouette): {cluster_analysis['optimal_clusters']}")



# Perform clustering with 4 clusters as requested
optimal_k = 4  # Fixed to 4 clusters as requested
cluster_labels = perform_clustering(scaled_features, optimal_k, algorithm='kmeans')

# Add cluster labels to original dataframe
students_df['cluster'] = cluster_labels

print(f"\n👥 Clustering Results:")
print(f"Number of clusters: {optimal_k} (fixed)")
print(f"Cluster distribution:")
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
for cluster, count in cluster_counts.items():
    print(f"  Cluster {cluster}: {count} students ({count/len(students_df)*100:.1f}%)")

display(students_df.groupby('cluster')[['communication_score', 'leadership_score', 
                                       'time_management_score', 'analytical_score', 'moyenne']].mean().round(2))

🔄 Preparing data for clustering analysis...
Features used for clustering: ['communication_score', 'leadership_score', 'time_management_score', 'analytical_score', 'extroversion_score', 'creativity_score', 'moyenne', 'extroversion_score', 'creativity_score', 'technical_confidence', 'specialization', 'preferred_programming_language', 'primary_learning_style', 'previous_group_experience']
Data shape: (120, 14)

🔍 Finding optimal number of clusters...
Optimal number of clusters (silhouette): 2

👥 Clustering Results:
Number of clusters: 4 (fixed)
Cluster distribution:
  Cluster 0: 24 students (20.0%)
  Cluster 1: 27 students (22.5%)
  Cluster 2: 37 students (30.8%)
  Cluster 3: 32 students (26.7%)


,communication_score,leadership_score,time_management_score,analytical_score,moyenne
cluster,,,,,
0,3.21,2.51,3.49,3.97,13.85
1,3.81,3.15,2.40,2.96,12.84
2,3.57,3.37,3.91,4.05,15.26
3,3.67,3.81,3.51,3.75,14.54


In [71]:
def create_heterogeneous_groups(df, group_size=5, strategy='balanced'):
    """
    Create heterogeneous student groups based on clustering results
    
    Args:
        df (pd.DataFrame): Student data with cluster assignments
        group_size (int): Target size for each group
        strategy (str): Group formation strategy ('balanced', 'diverse', 'mixed')
    
    Returns:
        pd.DataFrame: Students with group assignments
    """
    df_copy = df.copy()
    n_students = len(df_copy)
    n_groups = n_students // group_size
    
    if strategy == 'balanced':
        # Strategy 1: Balanced - Equal representation from each cluster
        groups = []
        cluster_counts = df_copy['cluster'].value_counts().sort_index()
        
        # Calculate how many students from each cluster per group
        students_per_cluster_per_group = {}
        for cluster in cluster_counts.index:
            students_per_cluster_per_group[cluster] = max(1, cluster_counts[cluster] // n_groups)
        
        for group_id in range(n_groups):
            group_students = []
            
            # Add students from each cluster
            for cluster in cluster_counts.index:
                cluster_students = df_copy[df_copy['cluster'] == cluster].sample(
                    min(students_per_cluster_per_group[cluster], 
                        len(df_copy[df_copy['cluster'] == cluster])),
                    random_state=42 + group_id
                )
                group_students.extend(cluster_students.index.tolist())
                df_copy = df_copy.drop(cluster_students.index)
            
            # Fill remaining spots randomly if needed
            remaining_spots = group_size - len(group_students)
            if remaining_spots > 0 and len(df_copy) > 0:
                additional_students = df_copy.sample(
                    min(remaining_spots, len(df_copy)), 
                    random_state=42 + group_id
                )
                group_students.extend(additional_students.index.tolist())
                df_copy = df_copy.drop(additional_students.index)
            
            groups.append(group_students[:group_size])
    
    elif strategy == 'diverse':
        # Strategy 2: Maximize diversity in each group
        groups = []
        remaining_students = df_copy.index.tolist()
        
        for group_id in range(n_groups):
            if len(remaining_students) < group_size:
                break
            
            group_students = []
            temp_df = df_copy.loc[remaining_students]
            
            # Start with most different students (different clusters)
            unique_clusters = temp_df['cluster'].unique()
            
            for cluster in unique_clusters[:group_size]:
                cluster_students = temp_df[temp_df['cluster'] == cluster]
                if len(cluster_students) > 0:
                    selected = cluster_students.sample(1, random_state=42 + group_id)
                    group_students.extend(selected.index.tolist())
                    remaining_students.remove(selected.index[0])
            
            # Fill remaining spots
            while len(group_students) < group_size and remaining_students:
                # Select student from different major/learning style if possible
                temp_df = df_copy.loc[remaining_students]
                current_specializations = df_copy.loc[group_students]['specialization'].unique()
                
                # Prefer students with different specializations
                different_specialization_students = temp_df[~temp_df['specialization'].isin(current_specializations)]
                if len(different_specialization_students) > 0:
                    selected = different_specialization_students.sample(1, random_state=42 + group_id)
                else:
                    selected = temp_df.sample(1, random_state=42 + group_id)
                
                group_students.append(selected.index[0])
                remaining_students.remove(selected.index[0])
            
            groups.append(group_students)
    
    # Assign group numbers to students
    df_result = df.copy()
    df_result['group'] = -1  # Unassigned
    
    for i, group in enumerate(groups):
        df_result.loc[group, 'group'] = i + 1
    
    # Handle remaining students
    unassigned = df_result[df_result['group'] == -1]
    if len(unassigned) > 0:
        # Distribute remaining students to existing groups
        for i, student_idx in enumerate(unassigned.index):
            target_group = (i % n_groups) + 1
            df_result.loc[student_idx, 'group'] = target_group
    
    return df_result

def analyze_group_diversity(df):
    """
    Analyze the diversity and balance of formed IT engineering student groups
    
    Args:
        df (pd.DataFrame): Student data with group assignments
    
    Returns:
        dict: Group analysis results
    """
    groups = df[df['group'] > 0]['group'].unique()
    analysis = {'groups': []}
    
    skill_cols = [col for col in df.columns if col.endswith('_score')]
    
    for group in sorted(groups):
        group_data = df[df['group'] == group]
        
        group_analysis = {
            'group_id': group,
            'size': len(group_data),
            'clusters_represented': len(group_data['cluster'].unique()),
            'specializations_represented': len(group_data['specialization'].unique()),
            'programming_languages': len(group_data['preferred_programming_language'].unique()),
            'learning_styles': len(group_data['primary_learning_style'].unique()),
            'avg_moyenne': round(group_data['moyenne'].mean(), 2),
            'moyenne_std': round(group_data['moyenne'].std(), 2),
            'skill_averages': {},
            'skill_diversity': {}
        }
        
        # Calculate skill averages and diversity
        for skill in skill_cols:
            group_analysis['skill_averages'][skill] = round(group_data[skill].mean(), 2)
            group_analysis['skill_diversity'][skill] = round(group_data[skill].std(), 2)
        
        analysis['groups'].append(group_analysis)
    
    # Overall statistics
    analysis['overall'] = {
        'total_groups': len(groups),
        'avg_group_size': round(np.mean([g['size'] for g in analysis['groups']]), 1),
        'avg_clusters_per_group': round(np.mean([g['clusters_represented'] for g in analysis['groups']]), 1),
        'avg_specializations_per_group': round(np.mean([g['specializations_represented'] for g in analysis['groups']]), 1)
    }
    
    return analysis

# Create heterogeneous groups
print("\n🎯 Creating heterogeneous IT engineering student groups...")
students_with_groups = create_heterogeneous_groups(students_df, group_size=5, strategy='diverse')

# Analyze group diversity
group_analysis = analyze_group_diversity(students_with_groups)

print(f"📊 Group Formation Results:")
print(f"Total groups created: {group_analysis['overall']['total_groups']}")
print(f"Average group size: {group_analysis['overall']['avg_group_size']}")
print(f"Average clusters per group: {group_analysis['overall']['avg_clusters_per_group']}")
print(f"Average specializations per group: {group_analysis['overall']['avg_specializations_per_group']}")

# Display sample groups
print(f"\n👥 Sample Groups:")
for i in range(min(3, group_analysis['overall']['total_groups'])):
    group_data = students_with_groups[students_with_groups['group'] == i + 1]
    print(f"\nGroupe {i + 1}:")  # French: Group
    print(f"  Étudiants: {', '.join(group_data['student_id'].tolist())}")  # French: Students
    print(f"  Clusters: {group_data['cluster'].tolist()}")
    print(f"  Spécialisations: {group_data['specialization'].tolist()}")  # French: Specializations
    print(f"  Archétypes: {group_data['archetype'].tolist()}")  # French: Archetypes
    print(f"  Moyenne moyenne: {group_data['moyenne'].mean():.2f}/20")  # French: Average moyenne
    
    skill_avgs = {}
    for skill in ['communication_score', 'leadership_score', 'time_management_score', 'analytical_score']:
        skill_avgs[skill.replace('_score', '')] = f"{group_data[skill].mean():.1f}"
    print(f"  Moyennes des compétences: {skill_avgs}")  # French: Skill averages

# Show group distribution by cluster
print(f"\n📈 Distribution Groupe-Cluster:")  # French: Group-Cluster Distribution
group_cluster_dist = students_with_groups.groupby(['group', 'cluster']).size().unstack(fill_value=0)
display(group_cluster_dist)


🎯 Creating heterogeneous IT engineering student groups...
📊 Group Formation Results:
Total groups created: 24
Average group size: 5.0
Average clusters per group: 3.7
Average specializations per group: 4.6

👥 Sample Groups:

Groupe 1:
  Étudiants: ETU_022, ETU_032, ETU_043, ETU_055, ETU_112
  Clusters: [0, 0, 1, 2, 3]
  Spécialisations: ['DevOps & Cloud Computing', 'Web Development', 'Cybersecurity', 'Network Engineering', 'Embedded Systems']
  Archétypes: ['Emerging Scholar', 'Analytical Thinker', 'Collaborative Communicator', 'Analytical Thinker', 'Emerging Scholar']
  Moyenne moyenne: 14.21/20
  Moyennes des compétences: {'communication': '3.8', 'leadership': '2.8', 'time_management': '3.5', 'analytical': '3.6'}

Groupe 2:
  Étudiants: ETU_020, ETU_029, ETU_045, ETU_104, ETU_114
  Clusters: [2, 2, 1, 3, 0]
  Spécialisations: ['Embedded Systems', 'Data Science & AI', 'Cybersecurity', 'Web Development', 'Network Engineering']
  Archétypes: ['Mixed Profile', 'Mixed Profile', 'Collabora

cluster,0,1,2,3
group,,,,
1,2,1,1,1
2,1,1,2,1
3,1,1,1,2
4,2,1,1,1
5,2,1,1,1
6,1,1,2,1
7,1,2,1,1
8,2,1,1,1
9,1,1,2,1


In [72]:
def visualize_clustering_analysis(cluster_analysis, students_df):
    """
    Create comprehensive visualizations for clustering analysis
    """
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Elbow Method for Optimal Clusters',
            'Silhouette Score Analysis', 
            'Student Distribution by Cluster',
            'Cluster Characteristics'
        ],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"type": "bar"}, {"type": "heatmap"}]]
    )
    
    # Elbow method plot
    fig.add_trace(
        go.Scatter(
            x=cluster_analysis['cluster_range'],
            y=cluster_analysis['inertias'],
            mode='lines+markers',
            name='Inertia',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    
    # Silhouette score plot
    fig.add_trace(
        go.Scatter(
            x=cluster_analysis['cluster_range'],
            y=cluster_analysis['silhouette_scores'],
            mode='lines+markers',
            name='Silhouette Score',
            line=dict(color='red')
        ),
        row=1, col=2
    )
    
    # Cluster distribution
    cluster_counts = students_df['cluster'].value_counts().sort_index()
    fig.add_trace(
        go.Bar(
            x=[f'Cluster {i}' for i in cluster_counts.index],
            y=cluster_counts.values,
            name='Student Count',
            marker_color='lightblue'
        ),
        row=2, col=1
    )
    
    # Cluster characteristics heatmap
    skill_cols = [col for col in students_df.columns if col.endswith('_score')]
    cluster_means = students_df.groupby('cluster')[skill_cols + ['moyenne']].mean()
    
    fig.add_trace(
        go.Heatmap(
            z=cluster_means.values,
            x=[col.replace('_score', '').title() for col in cluster_means.columns],
            y=[f'Cluster {i}' for i in cluster_means.index],
            colorscale='Viridis',
            name='Avg Score'
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        title_text="Clustering Analysis Results",
        showlegend=False
    )
    
    fig.show()

def visualize_group_formation(students_df, group_analysis):
    """
    Create visualizations for group formation results
    """
    # Create multiple visualizations
    
    # 1. Group composition overview
    fig1 = make_subplots(
        rows=1, cols=3,
        subplot_titles=['Group Sizes', 'Clusters per Group', 'Specializations per Group'],
        specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]]
    )
    
    groups_data = group_analysis['groups']
    group_ids = [g['group_id'] for g in groups_data]
    
    # Group sizes
    fig1.add_trace(
        go.Bar(x=group_ids, y=[g['size'] for g in groups_data], name='Size'),
        row=1, col=1
    )
    
    # Clusters per group
    fig1.add_trace(
        go.Bar(x=group_ids, y=[g['clusters_represented'] for g in groups_data], name='Clusters'),
        row=1, col=2
    )
    
    # Specializations per group
    fig1.add_trace(
        go.Bar(x=group_ids, y=[g['specializations_represented'] for g in groups_data], name='Specializations'),
        row=1, col=3
    )
    
    fig1.update_layout(height=400, title_text="Group Composition Overview", showlegend=False)
    fig1.show()
    
    # 2. Skill distribution across groups
    skill_cols = [col for col in students_df.columns if col.endswith('_score')]
    skill_names = [col.replace('_score', '').title() for col in skill_cols]
    
    fig2 = go.Figure()
    
    for i, skill in enumerate(skill_cols):
        skill_name = skill.replace('_score', '').title()
        group_avgs = [g['skill_averages'][skill] for g in groups_data]
        
        fig2.add_trace(go.Bar(
            name=skill_name,
            x=group_ids,
            y=group_avgs,
            offsetgroup=i
        ))
    
    fig2.update_layout(
        title="Average Skill Scores by Group",
        xaxis_title="Group ID",
        yaxis_title="Average Score",
        barmode='group',
        height=500
    )
    fig2.show()
    
    # 3. Group diversity analysis
    fig3 = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Moyenne Distribution', 'Cluster Distribution', 'Specialization Distribution', 'Learning Style Distribution']
    )
    
    # Moyenne distribution by group
    for group_id in group_ids[:5]:  # Show first 5 groups
        group_data = students_df[students_df['group'] == group_id]
        fig3.add_trace(
            go.Box(y=group_data['moyenne'], name=f'Group {group_id}'),
            row=1, col=1
        )
    
    # Cluster distribution heatmap
    group_cluster_dist = students_df.groupby(['group', 'cluster']).size().unstack(fill_value=0)
    fig3.add_trace(
        go.Heatmap(
            z=group_cluster_dist.values,
            x=[f'Cluster {i}' for i in group_cluster_dist.columns],
            y=[f'Group {i}' for i in group_cluster_dist.index],
            colorscale='Blues'
        ),
        row=1, col=2
    )
    
    # Specialization distribution
    specialization_dist = students_df.groupby(['group', 'specialization']).size().unstack(fill_value=0)
    fig3.add_trace(
        go.Heatmap(
            z=specialization_dist.values,
            x=specialization_dist.columns,
            y=[f'Group {i}' for i in specialization_dist.index],
            colorscale='Greens'
        ),
        row=2, col=1
    )
    
    # Learning style distribution
    style_dist = students_df.groupby(['group', 'primary_learning_style']).size().unstack(fill_value=0)
    fig3.add_trace(
        go.Heatmap(
            z=style_dist.values,
            x=style_dist.columns,
            y=[f'Group {i}' for i in style_dist.index],
            colorscale='Oranges'
        ),
        row=2, col=2
    )
    
    fig3.update_layout(height=800, title_text="Group Diversity Analysis", showlegend=False)
    fig3.show()

def create_student_profile_radar(student_data):
    """
    Create a radar chart for individual student profile
    """
    skills = ['Communication', 'Leadership', 'Time Management', 'Analytical']
    skill_cols = ['communication_score', 'leadership_score', 'time_management_score', 'analytical_score']
    values = [student_data[col] for col in skill_cols]
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=values + [values[0]],  # Close the polygon
        theta=skills + [skills[0]],
        fill='toself',
        name=f"Student {student_data['student_id']}"
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 5]
            )),
        title=f"Skill Profile: {student_data['student_id']} ({student_data['archetype']})",
        showlegend=True
    )
    
    return fig

# Generate visualizations
print("📊 Generating clustering and group formation visualizations...")

# Visualize clustering analysis
visualize_clustering_analysis(cluster_analysis, students_df)

# Visualize group formation results
visualize_group_formation(students_with_groups, group_analysis)

# Show sample student profiles
print("\n👤 Sample Student Profiles:")
sample_students = students_with_groups.sample(3, random_state=42)
for _, student in sample_students.iterrows():
    fig = create_student_profile_radar(student)
    fig.show()
    print(f"Student: {student['student_id']}, Cluster: {student['cluster']}, Group: {student['group']}")
    print(f"Archetype: {student['archetype']}, Specialization: {student['specialization']}, Moyenne: {student['moyenne']}")
    print("---")

📊 Generating clustering and group formation visualizations...



👤 Sample Student Profiles:


Student: ETU_045, Cluster: 1, Group: 2
Archetype: Collaborative Communicator, Specialization: Cybersecurity, Moyenne: 13.73
---


Student: ETU_048, Cluster: 2, Group: 23
Archetype: Mixed Profile, Specialization: Mobile Development, Moyenne: 14.58
---


Student: ETU_005, Cluster: 1, Group: 5
Archetype: Well-Rounded Student, Specialization: DevOps & Cloud Computing, Moyenne: 14.03
---


## 9. Practical Implementation: Web Application with Gamification

### Integration with Your Project Goals

This clustering analysis can be directly integrated into your web application for automatic heterogeneous group formation:

### **Web Application Features:**

1. **Student Assessment Interface**
   - Students complete soft skills assessment
   - Collect additional data: moyenne, specialization, learning style preferences
   - User-friendly questionnaire with progress tracking

2. **Automatic Group Formation**
   - Apply clustering algorithms in real-time
   - Create balanced groups based on diversity metrics
   - Ensure equal representation across skill levels

3. **Gamification Elements**
   - **Avatar Selection**: Students choose personalized avatars
   - **Progress Badges**: Earn badges for assessment completion, group participation
   - **Skill Visualization**: Radar charts showing individual and group profiles
   - **Group Challenges**: Collaborative tasks that leverage diverse skills

### **Implementation Steps:**

1. **Data Collection Layer**
   ```python
   # Collect student data through web forms
   student_data = {
       'soft_skills': assessment_scores,
       'academic': {'moyenne': moyenne, 'specialization': specialization},
       'preferences': {'learning_style': style, 'interests': interests},
       'gamification': {'avatar': avatar_choice, 'badges': earned_badges}
   }
   ```

2. **Clustering Pipeline**
   ```python
   # Real-time clustering and group formation
   clusters = perform_clustering(student_data)
   groups = create_heterogeneous_groups(students_with_clusters)
   ```

3. **Visualization Dashboard**
   - Interactive group composition displays
   - Skill distribution charts
   - Progress tracking and feedback

### **Benefits for Educational Outcomes:**

- **Reduced Bias**: Eliminates instructor preferences in group formation
- **Enhanced Learning**: Diverse perspectives improve critical thinking
- **Improved Engagement**: Gamification increases participation
- **Data-Driven Insights**: Analytics on group performance and dynamics
- **Scalability**: Handles large class sizes automatically

### **Next Steps for Your Project:**

1. Build web interface using Streamlit/Flask/Django
2. Integrate clustering algorithms as backend services
3. Implement gamification features and avatar system
4. Add real-time group formation and visualization
5. Create instructor dashboard for monitoring and analytics

In [73]:
def export_group_assignments(students_df, filename='student_group_assignments.csv'):
    """
    Export group assignments for use in web application
    """
    # Select relevant columns for export
    export_columns = [
        'student_id', 'group', 'cluster', 'archetype',
        'communication_score', 'leadership_score', 'time_management_score', 'analytical_score',
        'moyenne', 'specialization', 'preferred_programming_language', 'primary_learning_style', 'primary_interest'
    ]
    
    export_df = students_df[export_columns].copy()
    export_df = export_df[export_df['group'] > 0]  # Only assigned students
    
    # Sort by group and then by student ID
    export_df = export_df.sort_values(['group', 'student_id'])
    
    # Save to CSV
    export_df.to_csv(filename, index=False)
    print(f"✅ Group assignments exported to: {filename}")
    
    return export_df

def generate_group_summary_report(students_df):
    """
    Generate a comprehensive summary report for instructors
    """
    report = {
        'total_students': len(students_df),
        'total_groups': len(students_df[students_df['group'] > 0]['group'].unique()),
        'clustering_summary': {},
        'diversity_metrics': {},
        'group_details': []
    }
    
    # Clustering summary
    cluster_distribution = students_df['cluster'].value_counts().to_dict()
    report['clustering_summary'] = {
        'num_clusters': len(cluster_distribution),
        'cluster_distribution': cluster_distribution,
        'largest_cluster': max(cluster_distribution.values()),
        'smallest_cluster': min(cluster_distribution.values())
    }
    
    # Diversity metrics
    assigned_students = students_df[students_df['group'] > 0]
    report['diversity_metrics'] = {
        'avg_moyenne_range': round(assigned_students['moyenne'].max() - assigned_students['moyenne'].min(), 2),
        'unique_specializations': len(assigned_students['specialization'].unique()),
        'unique_programming_languages': len(assigned_students['preferred_programming_language'].unique()),
        'unique_learning_styles': len(assigned_students['primary_learning_style'].unique()),
        'skill_score_ranges': {}
    }
    
    # Skill score ranges
    skill_cols = [col for col in students_df.columns if col.endswith('_score')]
    for skill in skill_cols:
        skill_name = skill.replace('_score', '')
        report['diversity_metrics']['skill_score_ranges'][skill_name] = {
            'min': round(assigned_students[skill].min(), 2),
            'max': round(assigned_students[skill].max(), 2),
            'range': round(assigned_students[skill].max() - assigned_students[skill].min(), 2)
        }
    
    # Group details
    for group_id in sorted(assigned_students['group'].unique()):
        group_data = assigned_students[assigned_students['group'] == group_id]
        
        group_detail = {
            'group_id': group_id,
            'size': len(group_data),
            'students': group_data['student_id'].tolist(),
            'clusters_represented': len(group_data['cluster'].unique()),
            'specializations': group_data['specialization'].unique().tolist(),
            'programming_languages': group_data['preferred_programming_language'].unique().tolist(),
            'avg_moyenne': round(group_data['moyenne'].mean(), 2),
            'skill_averages': {}
        }
        
        for skill in skill_cols:
            skill_name = skill.replace('_score', '')
            group_detail['skill_averages'][skill_name] = round(group_data[skill].mean(), 2)
        
        report['group_details'].append(group_detail)
    
    return report

# Export results for web application
print("💾 Exporting Results for Web Application Integration...")

# Export group assignments
exported_data = export_group_assignments(students_with_groups, 'data/student_group_assignments.csv')

# Generate summary report
summary_report = generate_group_summary_report(students_with_groups)

print(f"\n📋 Summary Report:")
print(f"Total Students: {summary_report['total_students']}")
print(f"Total Groups: {summary_report['total_groups']}")
print(f"Clusters Identified: {summary_report['clustering_summary']['num_clusters']}")
print(f"Average Students per Group: {summary_report['total_students'] / summary_report['total_groups']:.1f}")

print(f"\n🎯 Diversity Achieved:")
print(f"Unique Specializations: {summary_report['diversity_metrics']['unique_specializations']}")
print(f"Unique Programming Languages: {summary_report['diversity_metrics']['unique_programming_languages']}")
print(f"Unique Learning Styles: {summary_report['diversity_metrics']['unique_learning_styles']}")
print(f"Moyenne Range per Class: {summary_report['diversity_metrics']['avg_moyenne_range']}")

print(f"\n📊 Sample Group Analysis:")
for i in range(min(3, len(summary_report['group_details']))):
    group = summary_report['group_details'][i]
    print(f"\nGroup {group['group_id']}:")
    print(f"  Size: {group['size']} students")
    print(f"  Clusters: {group['clusters_represented']} different clusters")
    print(f"  Specializations: {group['specializations']}")
    print(f"  Programming Languages: {group['programming_languages']}")
    print(f"  Avg Moyenne: {group['avg_moyenne']}")
    print(f"  Skill Balance: {group['skill_averages']}")

print(f"\n🚀 Ready for Web Application Integration!")
print(f"✅ Student assessment data: Generated and validated")
print(f"✅ Clustering algorithms: Implemented and tested") 
print(f"✅ Group formation: Automated and optimized")
print(f"✅ Diversity metrics: Calculated and verified")
print(f"✅ Export functionality: Available for web app")
print(f"✅ Visualization tools: Ready for dashboard")

print(f"\n🎮 Next Steps for Gamification:")
print(f"1. Integrate avatar selection system")
print(f"2. Implement badge and achievement tracking")
print(f"3. Create group challenge and collaboration features")
print(f"4. Add real-time progress visualization")
print(f"5. Build instructor dashboard for monitoring")

# Save the processed data
students_with_groups.to_csv('data/complete_student_analysis.csv', index=False)
print(f"\n💾 Complete analysis saved to: data/complete_student_analysis.csv")

💾 Exporting Results for Web Application Integration...
✅ Group assignments exported to: data/student_group_assignments.csv

📋 Summary Report:
Total Students: 120
Total Groups: 24
Clusters Identified: 4
Average Students per Group: 5.0

🎯 Diversity Achieved:
Unique Specializations: 8
Unique Programming Languages: 8
Unique Learning Styles: 7
Moyenne Range per Class: 8.56

📊 Sample Group Analysis:

Group 1:
  Size: 5 students
  Clusters: 4 different clusters
  Specializations: ['DevOps & Cloud Computing', 'Web Development', 'Cybersecurity', 'Network Engineering', 'Embedded Systems']
  Programming Languages: ['JavaScript', 'C#', 'React', 'PHP']
  Avg Moyenne: 14.21
  Skill Balance: {'communication': 3.82, 'leadership': 2.83, 'time_management': 3.49, 'analytical': 3.64, 'extroversion': 3.09, 'creativity': 2.87}

Group 2:
  Size: 5 students
  Clusters: 4 different clusters
  Specializations: ['Embedded Systems', 'Data Science & AI', 'Cybersecurity', 'Web Development', 'Network Engineering']
 